In [2]:
import torch
import numpy as np
from torch import nn
from utils.model import Spice_model
from torchinfo import summary
import pandas as pd
from data_files.dataloader import MedleyDBLoader
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from utils.model import Spice_model
from utils.training_script import Trainer
from optims.loss import Huber_loss, Recons_loss, Conf_loss
from data_files.dataset import CQT_Dataset

2023-07-25 20:25:22.265305: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 20:25:22.306813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 20:25:22.833794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(torch.__version__)
torch.manual_seed(191009)

2.0.1


In [4]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

In [15]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell2(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell2, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forwardMyDecisionGate(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

In [5]:
my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[ 0.3179, -0.0809, -0.6178,  0.3956],
        [ 0.6665,  0.1408,  0.4307,  0.7632],
        [ 0.5047, -0.2405, -0.1733,  0.6841]], grad_fn=<TanhBackward0>), tensor([[ 0.3179, -0.0809, -0.6178,  0.3956],
        [ 0.6665,  0.1408,  0.4307,  0.7632],
        [ 0.5047, -0.2405, -0.1733,  0.6841]], grad_fn=<TanhBackward0>))


In [6]:
my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print("tracd",traced_cell,"end")
traced_cell(x, h)

tracd MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
) end


(tensor([[-0.0416,  0.7165,  0.5299, -0.0434],
         [ 0.2687,  0.1412,  0.6382,  0.1054],
         [ 0.3480,  0.5014,  0.6016, -0.3498]], grad_fn=<TanhBackward0>),
 tensor([[-0.0416,  0.7165,  0.5299, -0.0434],
         [ 0.2687,  0.1412,  0.6382,  0.1054],
         [ 0.3480,  0.5014,  0.6016, -0.3498]], grad_fn=<TanhBackward0>))

In [9]:
#print(traced_cell.graph)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



In [16]:
my_cell = MyCell2(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



/tmp/ipykernel_394348/3990843855.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [18]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell2(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)
print(scripted_cell(x, h))

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)

(tensor([[0.0638, 0.9258, 0.5977, 0.2638],
        [0.2533, 0.6691, 0.6879, 0.5501],
        [0.5337, 0.8923, 0.7698, 0.2902]], grad_fn=<TanhBackward0>), tensor([[0.0638, 0.9258, 0.5977, 0.2638],
        [0.2533, 0.6691, 0.6879, 0.5501],
        [0.5337, 0.8923, 0.7698, 0.2902]], grad_fn=<TanhBackward0>))


In [21]:

class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell2(scripted_gate), (x, h))
    @torch.jit.script
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = MyRNNLoop()
print(rnn_loop.code)

RuntimeError: Type '<class '__main__.MyRNNLoop'>' cannot be compiled since it inherits from nn.Module, pass an instance instead

In [ ]:
fs = 16000
med = MedleyDBLoader(fs )

In [5]:
f1 = 1110.0644999999954 
f2 = -5.684341886080802e-14
dive = f1/f2
dive

-1.9528461205301544e+16

In [6]:
np.log2(dive)

/tmp/ipykernel_23855/3868652844.py:1: RuntimeWarning: invalid value encountered in log2
  np.log2(dive)


nan

In [3]:
data_pd = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [6]:
nnp = np.load('./CQT_data/MedleyDB.npy')

In [ ]:
nnp = torch.toTe(nnp)

In [4]:
data_pd.shape

(3530, 191)

In [7]:
#data_pd = pd.DataFrame(data=nnp) 
train, val = train_test_split(data_pd, train_size=0.8, test_size=0.2, random_state=1)
print("train shape: ", train.shape)
train_batches = DataLoader(CQT_Dataset(data=train, mode='train'), batch_size=64, shuffle=True)
print("train_batch NUmber: ", len(train_batches))
diff, slice1, slice2, f0 = next(iter(train_batches))
print(f"diff batch shape: {diff.size()}")
print(f"slice1 batch shape: {slice1.size()}")
print(f"slice2 batch shape: {f0.size()}")

spice = Spice_model()


for b in train_batches:
    print("batch shape:", b[0].shape)
    pitch_diff, x_1, x_2, f0 = b
    x_1 = x_1.type(torch.FloatTensor)
    #print(x_1.shape, x_1.type())
    a, x, y = spice(x_1)
    print(a.size(), x.size(), y.size())

train shape:  (2824, 191)
train_batch NUmber:  45
diff batch shape: torch.Size([64])
slice1 batch shape: torch.Size([64, 128])
slice2 batch shape: torch.Size([64])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64

In [6]:
xx=torch.arange(0, 25, 1)

In [12]:
xx = xx.reshape((5,-1))

In [13]:
xx.size()

torch.Size([5, 5])

In [ ]:
df = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [ ]:
df.shape

In [ ]:
np.arange(0,25,1).reshape(5,5)[:,-1:]

In [ ]:
a = np.arange(0,15,1).reshape(5,3)
a = np.hstack((a, np.array([0.25,0.7,0,0.5,0]).reshape(-1,1)))
a

In [ ]:
df = pd.DataFrame(a)
df.shape

In [ ]:
# remove rows of cqt where label (last) column is zero
df.drop(df.loc[df.iloc[:, -1]==0].index, inplace=True)
df.shape

In [ ]:
np.min(df.iloc[:, -1])

In [14]:
s = Spice_model()
summary(s, input_size=[10, 128])

Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
no unpooling True True
no unpooling False True
no unpooling False True
no unpooling False True
no unpooling False True


Layer (type:depth-idx)                   Output Shape              Param #
Spice_model                              [10, 1]                   --
├─Spice_Encoder: 1-1                     [10, 1]                   --
│    └─Conv_block: 2-1                   [10, 64, 64]              --
│    │    └─Conv1d: 3-1                  [10, 64, 128]             256
│    │    └─BatchNorm1d: 3-2             [10, 64, 128]             128
│    │    └─ReLU: 3-3                    [10, 64, 128]             --
│    │    └─MaxPool1d: 3-4               [10, 64, 64]              --
│    └─Conv_block: 2-2                   [10, 128, 32]             --
│    │    └─Conv1d: 3-5                  [10, 128, 64]             24,704
│    │    └─BatchNorm1d: 3-6             [10, 128, 64]             256
│    │    └─ReLU: 3-7                    [10, 128, 64]             --
│    │    └─MaxPool1d: 3-8               [10, 128, 32]             --
│    └─Conv_block: 2-3                   [10, 256, 16]             --
│    │  

In [24]:
for n, param in s.named_parameters():
    print(type(param), param.requires_grad)

<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Param